# Doubly Robust法(DR法)による因果推論の実装

In [1]:
# 乱数シードを固定
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.random import randint
from scipy.special import expit
from numpy.random import randn

random.seed(1234)
np.random.seed(1234)

In [2]:
# データ数
num_data = 200

# 年齢
x_1 = randint(15, 75, num_data) # 15から75までの一様乱数

# 性別
x_2 = randint(0, 2, num_data) # 0か1の乱数

# ノイズの発生
e_z = randn(num_data) # 平均0、標準偏差1の正規分布

# シグモイド関数に入れる部分
z_base = x_1 + (1 - x_2) * 10 - 40 + 5*e_z

# シグモイド関数を計算(CMを見る確率)
z_prob = expit(0.1 * z_base)


# テレビCMを見たかどうかの変数(0は見ていない、1は見た)
Z = np.array([])

for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p = [1 - z_prob[i], z_prob[i]])[0] # CMを見る確率をweightとして0　or　1を選ぶ
    Z = np.append(Z, Z_i)
    
# ノイズの発生
e_y = randn(num_data)

Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

df = pd.DataFrame({
    "年齢" : x_1,
    "性別" : x_2, 
    "CMを見た" : Z,
    "購入量" : Y
})

In [3]:
df.head()

,年齢,性別,CMを見た,購入量
0,62,1,1.0,88.634641
1,34,1,0.0,79.047707
2,53,0,1.0,29.331802
3,68,0,1.0,13.211139
4,27,0,1.0,64.056204


## 反実仮想の回帰モデル  
全員に対して  
- cmを見ていない(z=0)時の購入量   と　　
- cmを見た(z=1)時の購入量  
  
を求める

In [4]:
from sklearn.linear_model import LinearRegression

# 説明変数
X = df[["年齢", "性別", "CMを見た"]]

# 目的変数
y = df["購入量"]

In [5]:
# 回帰の実施(反実仮想)
reg2 = LinearRegression().fit(X, y)

In [6]:
# z= 0の場合
X_0 = X.copy()
X_0["CMを見た"] = 0
Y_0 =  reg2.predict(X_0)

# z= 1の場合
X_1 = X.copy()
X_1["CMを見た"] = 1
Y_1 =  reg2.predict(X_1)

### 傾向スコアを求めるロジスティック回帰モデルを構築

In [7]:
from sklearn.linear_model import LogisticRegression

# 説明変数
X = df[["年齢", "性別"]]

# 目的変数
Z = df["CMを見た"]

# 回帰の実施
reg = LogisticRegression().fit(X, Z)

# 傾向スコアを求める
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])

[[0.08420146 0.91579854]
 [0.53505367 0.46494633]
 [0.08392309 0.91607691]
 [0.02311324 0.97688676]
 [0.48908363 0.51091637]]


### ATEの実装

In [8]:
ATE_1_i = Y / Z_pre[:,1]*Z + (1-Z / Z_pre[:, 1])*Y_1
ATE_0_i = Y / Z_pre[:,0]*(1-Z) + (1 - (1-Z )/ Z_pre[:, 0])*Y_0
ATE = 1 / len(Y) * (ATE_1_i - ATE_0_i).sum()

print("推定したATE", ATE)

推定したATE 6.920851435108029
